In [1]:
from matplotlib import pyplot as plt
from statistics import mean
from statistics import stdev
from statistics import median
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy import stats
import pandas as pd
import statsmodels.formula.api as sm
import statsmodels.stats.sandwich_covariance as sw
import numpy as np
import statsmodels as statsmodels
import statsmodels.api as smf
from statsmodels.graphics import tsaplots
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Reading in daily USD data
xls_file = pd.ExcelFile('statistical_tests.xlsx')

# Parsing data
full_ret = xls_file.parse('full')
sub_ret = xls_file.parse('sub')

In [3]:
sub_ret

,dates,SubCIRCoby5,SubCIRCoby20,SubVasiBFGS5,SubVasiBFGS20,SubCIRCoby5Mod,SubVasiCoby5Mod
0,2015-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2015-02,0.067770,0.067770,-0.077675,0.000000,0.095792,-0.089187
2,2015-03,-0.006709,-0.006709,-0.002788,0.000000,-0.013896,0.000000
3,2015-04,0.057008,0.057008,-0.045938,0.000000,0.062438,0.072480
4,2015-05,0.019959,0.019959,-0.017469,-0.027396,0.020722,0.023633
...,...,...,...,...,...,...,...
79,2021-08,0.003050,0.008930,-0.015014,-0.015014,-0.000726,0.009112
80,2021-09,0.026144,0.066832,-0.068537,-0.068537,0.028329,0.027463
81,2021-10,-0.041395,-0.061696,0.074084,0.074084,-0.041405,-0.040620
82,2021-11,-0.013864,-0.015744,0.010180,0.010180,-0.012727,-0.016299


In [4]:
full_ret_test = sub_ret['SubCIRCoby5Mod'] # THIS NEEDS TO BE ADJUSTED
full_ret_test = pd.array(full_ret_test[1:])

In [5]:
len(full_ret_test)

83

In [6]:
bpsmult = 10000

In [7]:
mean_ret = mean(full_ret_test)
mean_ret * bpsmult

14.220001150302005

In [8]:
# Test statistics with HAC robust standard errors
mkt_data = pd.DataFrame(full_ret_test, columns=['rets'])
#one = np.ones(len(monthly_mean_ret))
mkt_data['one'] = 1
model = sm.ols( formula = 'rets ~ one',data = mkt_data).fit(cov_type='HAC', cov_kwds={'maxlags': 1})
#model = sm.ols( formula = 'rets ~ one',data = mkt_data).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   rets   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                   0.07648
Date:                Sun, 24 Jul 2022   Prob (F-statistic):              0.783
Time:                        16:18:23   Log-Likelihood:                 143.76
No. Observations:                  83   AIC:                            -285.5
Df Residuals:                      82   BIC:                            -283.1
Df Model:                           0                                         
Covariance Type:                  HAC                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0007      0.003      0.277      0.782      -0.004       0.006
one            0.0007      0.003      0.277      0.782      -0.004       0.006
==============================================================================
Omnibus:                       13.843   Durbin-Watson:                   1.545
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               30.168
Skew:                           0.504   Prob(JB):                     2.81e-07
Kurtosis:                       5.776   Cond. No.                     8.71e+15
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 1 lags and without small sample correction
[2] The smallest eigenvalue is 2.19e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [9]:
n = model.nobs
n

83.0

**Confidence interval for mean**

In [10]:
# HAC robust standard errors in basis points
HAC = model.bse[0]
HAC * bpsmult

25.709282652651133

In [11]:
# Lower bound for mean confidence interval in basis points
conf_mult = 1.96
(mean_ret - conf_mult * HAC) * bpsmult

-36.17019284889422

In [12]:
# Upper bound for mean confidence interval in basis points
(mean_ret + conf_mult * HAC) * bpsmult

64.61019514949822

In [13]:
#Margin of error in basis points
(conf_mult * HAC) * bpsmult

50.39019399919622

**T-statistic, noncorrected skew and noncorrected kurtosis**

In [14]:
# T-statistic
mean_ret / HAC

0.5531076593004683

In [15]:
# Non-corr skewness
skew(full_ret_test)

0.503614459452369

In [16]:
# Non-corr kurtosis
kurtosis(full_ret_test, fisher=False)

5.776487641301784

**Non corrected standard error and t-statistic**

In [17]:
# Non-corr standard error in basis points
(stdev(full_ret_test)/np.sqrt(n)) * bpsmult

47.27574808629657

In [18]:
# Backup checking for standard error
from scipy.stats import sem
sem(full_ret_test)

0.004727574808629658

In [19]:
# Non-corr t-statistic
mean_ret /(stdev(full_ret_test)/np.sqrt(n))

0.30078849570703753

In [20]:
# Backup checking for non-corrected t-stat
stats.ttest_1samp(full_ret_test, popmean=0, axis=None)

Ttest_1sampResult(statistic=0.30078849570703753, pvalue=0.7643371182262378)

**Confidence interval for skewness**

In [21]:
# Margin of errro for skewness confidence interval
skew_margin = conf_mult * np.sqrt((6*n*(n-1))/((n-2)*(n+1)*(n+3)))
skew_margin

0.5177812156787753

In [26]:
# lower bound for skewness confidence interval. SKEW NEEDS TO BE MANUALLY ADJUSTED
0.504 - skew_margin

-0.013781215678775327

In [27]:
# Upper bound for skewness confidence interval. SKEW NEEDS TO BE MANUALLY ADJUSTED
0.504 + skew_margin

1.0217812156787753

**Noncorr LB, UB for mean of returns**

In [53]:
mean_noncorr = 14.220
stderror_noncorr = 47.27575

In [54]:
#Low bound
mean_noncorr - conf_mult * stderror_noncorr

-78.44047

In [55]:
#Upper bound
mean_noncorr + conf_mult * stderror_noncorr

106.88047

In [24]:
'''
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
DescTools = importr('DescTools')
""" You will probably need this if you want to work with numpy arrays"""
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()


def compute_skew(data, confidence_level=0.99):
    """ Compute the skew and confidence interval using rpy2, DescTools
        @param data
        @return dict with keys: skew, skew_ci_lower, skew_ci_upper"""
    d = {}
    d["skew"], d["skew_ci_lower"], d["skew_ci_upper"] = DescTools.Skew(data, conf_level=confidence_level)
    return d

""" Call the function on your data (assuming that is saved in a variable named sample)"""
print(compute_skew(sample))
'''

'\nimport rpy2.robjects as robjects\nfrom rpy2.robjects.packages import importr\nDescTools = importr(\'DescTools\')\n""" You will probably need this if you want to work with numpy arrays"""\nimport rpy2.robjects.numpy2ri\nrpy2.robjects.numpy2ri.activate()\n\n\ndef compute_skew(data, confidence_level=0.99):\n    """ Compute the skew and confidence interval using rpy2, DescTools\n        @param data\n        @return dict with keys: skew, skew_ci_lower, skew_ci_upper"""\n    d = {}\n    d["skew"], d["skew_ci_lower"], d["skew_ci_upper"] = DescTools.Skew(data, conf_level=confidence_level)\n    return d\n\n""" Call the function on your data (assuming that is saved in a variable named sample)"""\nprint(compute_skew(sample))\n'

In [25]:
'''
model = sm.ols( formula = 'rets ~ one',data = mkt_data).fit()
new = model.get_robustcov_results(cov_type='HAC',maxlags=1)
T_test = model.t_test(new.params) 
print(T_test)
'''

"\nmodel = sm.ols( formula = 'rets ~ one',data = mkt_data).fit()\nnew = model.get_robustcov_results(cov_type='HAC',maxlags=1)\nT_test = model.t_test(new.params) \nprint(T_test)\n"